<a href="https://colab.research.google.com/github/zainmo11/BERT_from_scratch_using_JAX-flax/blob/main/BERT_from_scratch_using_JAX%26fla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirement

In [ ]:
!pip install jax


In [42]:
!pip install --quiet flax
import jax
from flax import linen as jnn

# important Imports

In [43]:
import math
import jax
import jax.numpy as jnp
import jax.nn as jnn
import optax
import re
from random import *
import jax.scipy.special as jsp

# Text data example

In [44]:
text = (
    'I enjoy solving complex problems and the satisfaction that comes with finding innovative solutions. There’s something exhilarating about coding, isn’t there? Whether it\'s debugging a tricky issue or designing a new feature from scratch, the process of problem-solving is incredibly rewarding. I can spend hours immersed in code, trying to understand how different components interact and how to optimize their performance. However, I can\'t stand inefficiency and poorly designed systems that make even the simplest tasks cumbersome. Dealing with outdated documentation is particularly frustrating, especially when it hinders progress and wastes valuable time.\n'

    'One of the aspects I find most thrilling about programming is the ability to refactor and improve existing code. The challenge of taking something old and making it better, more efficient, or more readable is immensely satisfying. It’s like giving a new lease on life to a piece of software that might otherwise be languishing in obscurity. On the other hand, I dislike dealing with codebases that are full of hacks and workarounds rather than clean, maintainable solutions.\n'

    'Exploring new algorithms and data structures is another aspect of programming that I deeply enjoy. The way different algorithms can be applied to solve various types of problems is fascinating. It’s a never-ending learning experience, as there’s always something new to discover or a different approach to consider. I find that the flexibility and adaptability of algorithms make them incredibly powerful tools in software development. However, I do have a strong aversion to systems that lack proper documentation or are poorly maintained. It can be quite challenging to work with legacy systems that are difficult to understand or modify.\n'

    'Building something from the ground up, whether it\'s a new application or a piece of infrastructure, is always an exciting challenge. The process of taking an idea and turning it into a working product is both demanding and rewarding. It requires careful planning, design, and execution, and the sense of accomplishment when the project is completed is unparalleled. I find that this sense of achievement is one of the main reasons I continue to be passionate about programming.\n'

    'Despite my enthusiasm, I do encounter frustrations along the way. For instance, when the system crashes unexpectedly or when features are removed without prior notice, it can be quite disheartening. I hate dealing with such disruptions, as they can throw off development schedules and require additional troubleshooting. Compatibility issues are another source of frustration. When different components or systems don’t work well together, it can lead to significant delays and additional work.\n'

    'Integrating APIs and external services into a project can be both a challenge and an opportunity. On one hand, it can be tricky to get everything to work seamlessly together, especially when dealing with inconsistencies or unexpected behavior. On the other hand, it provides an opportunity to extend the functionality of a project and leverage existing tools and services. This balancing act between integration and customization is an essential part of modern software development.\n'

    'Testing is another critical area of focus. While it can sometimes be tedious and time-consuming, it is essential for ensuring that the software works as expected and is free of critical bugs. The process of writing and running tests, analyzing the results, and making necessary adjustments is a fundamental part of the development lifecycle. I often find that the effort put into testing pays off in the long run by preventing issues and improving the overall quality of the software.\n'

    'Collaboration is also an integral part of the development process. Working with other developers, designers, and stakeholders helps to ensure that a project meets its requirements and is aligned with its goals. While collaborating with different teams can be challenging, especially when dealing with diverse opinions and approaches, it also provides valuable opportunities for learning and growth. Effective communication and teamwork are essential for the success of any project, and I appreciate the opportunity to work with talented individuals from various backgrounds.\n'

    'Overall, programming is a field full of challenges and rewards. The constant evolution of technology and the need to stay up-to-date with the latest tools and practices keep the work exciting and dynamic. Despite the occasional frustrations and setbacks, the sense of accomplishment and the opportunity to create meaningful and impactful solutions make it all worthwhile. I look forward to continuing to explore new technologies, tackle complex problems, and contribute to the ever-evolving world of software development. The journey of learning and growing as a developer is something I cherish, and I am excited for the future opportunities and challenges that lie ahead.'
)


## 3- Data Preprocessing

1. Clean Data from .,?!
2. Encode our data (ids).

In [45]:
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'

word_list = list(set(" ".join(sentences).split())) # we make a join for all sentence to be as one sentense then split by " " and make it in set to remove redundant words the in a list
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


for i, w in enumerate(word_list):
    word_dict[w] = i + 4          # For Encoding start from 4

number_dict = {i: w for i, w in enumerate(word_dict)} # Create a reverse dictionary for decoding
vocab_size = len(word_dict)

token_list = list()
# Tokenize each sentence into a list of token IDs
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    # print("arr = ", arr)
    token_list.append(arr)

print (token_list)

[[236, 194, 62, 173, 170, 100, 295, 55, 352, 108, 29, 132, 276, 336, 164, 200, 239, 294, 272, 14, 144, 216, 326, 79, 197, 266, 288, 116, 240, 197, 163, 212, 233, 19, 295, 290, 147, 161, 257, 269, 90, 236, 92, 57, 254, 25, 209, 8, 69, 45, 86, 309, 252, 50, 133, 100, 309, 45, 192, 199, 88, 207, 236, 282, 156, 77, 100, 342, 10, 183, 352, 223, 291, 295, 324, 111, 317, 153, 29, 119, 243, 257, 158, 330, 102, 308, 345, 348, 224, 100, 76, 83, 35], [334, 147, 295, 51, 236, 248, 206, 357, 294, 268, 257, 295, 260, 45, 73, 100, 56, 293, 8, 295, 210, 147, 307, 200, 339, 100, 71, 345, 101, 117, 85, 116, 117, 184, 257, 80, 139, 198, 275, 211, 197, 163, 344, 182, 299, 45, 197, 225, 147, 30, 352, 9, 217, 281, 148, 209, 104, 182, 295, 310, 188, 236, 329, 153, 29, 95, 352, 128, 174, 147, 6, 100, 346, 123, 15, 323, 235, 336], [202, 163, 126, 100, 7, 106, 257, 259, 253, 147, 268, 352, 236, 189, 194, 295, 327, 252, 126, 92, 281, 205, 45, 122, 358, 286, 147, 170, 257, 39, 198, 197, 353, 347, 234, 331, 164, 3

# 4. Prepare our Data for BERT.
1. Configure BERT Parameters.
2. Make Batches.

In [46]:
batch_size = 6
maxlen = 30 # maximum of length
max_pred = 5  # max tokens of prediction
n_layers = 6 # number of Encoder of Encoder Layer
n_heads = 12 # number of heads in Multi-Head Attention
d_model = 768 # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V        64*12 head = 768
n_segments = 2 # n.o.sentences

In [47]:
def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))  # random index , random index
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]  # random sentence , random sentence

        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']] # concatenate random sentences with 'CLS' and 'SEP'

        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)  # [ 0,0,0,0,0,0,1,1,1,1,1,1,1,1,1]

        #MASK LM
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # number of masks is up to 15 % of tokens in one sentence

        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]  # list of indices of input without cls and sep tokens

        shuffle(cand_maked_pos)    # shuffle all indices
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)     # list of random indices
            masked_tokens.append(input_ids[pos]) # list of random tokens
            if random() < 0.8:  # 80%
                input_ids[pos] = word_dict['[MASK]'] # make mask
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace by random word

        # Zero Paddings
        n_pad = maxlen - len(input_ids) # number of pad tokens we need
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

    #     # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred: # target 1 , max = 5   [12,0,0,0,0 ]
            n_pad = max_pred - n_pred # number of pad tokens in masked list
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)  # to make masked list is static in shape
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch

# 5. Core Functions
1. Get Attention Pad Mask.
2. GELU.

In [54]:
# Function to get attention mask
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.shape
    _, len_k = seq_k.shape
    # Create padding mask where 0 is the padding
    pad_attn_mask = (seq_k == 0).astype(jnp.float32)
    # Expand dimensions for broadcasting
    pad_attn_mask = pad_attn_mask[:, jnp.newaxis, :]
    pad_attn_mask = jnp.broadcast_to(pad_attn_mask, (batch_size, len_q, len_k))
    return pad_attn_mask


In [49]:
def gelu(x):
    return x * 0.5 * (1.0 + jsp.erf(x / jnp.sqrt(2.0)))

In [50]:
batch = make_batch()
print(batch)

[[[1, 97, 28, 149, 236, 105, 297, 271, 226, 295, 327, 340, 325, 308, 295, 129, 328, 3, 116, 308, 115, 128, 313, 135, 301, 109, 345, 92, 281, 273, 218, 236, 283, 153, 29, 3, 44, 331, 284, 92, 34, 72, 87, 175, 100, 52, 318, 180, 181, 32, 128, 259, 114, 147, 91, 308, 3, 50, 116, 183, 81, 165, 355, 196, 345, 92, 351, 45, 179, 238, 100, 318, 165, 2, 23, 268, 257, 197, 267, 174, 147, 191, 100, 279, 295, 75, 280, 147, 316, 100, 295, 4, 45, 220, 136, 29, 295, 246, 124, 100, 176, 255, 295, 165, 305, 100, 190, 97, 295, 112, 271, 100, 18, 295, 337, 147, 208, 100, 295, 264, 45, 84, 249, 100, 169, 336, 223, 345, 12, 125, 236, 356, 49, 45, 277, 45, 74, 163, 157, 172, 173, 170, 100, 107, 45, 295, 247, 120, 147, 30, 87, 295, 304, 147, 347, 100, 321, 331, 197, 168, 257, 200, 236, 3, 100, 236, 193, 229, 340, 295, 131, 66, 100, 191, 352, 127, 341, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [55]:
# Function to pad sequences
def pad_sequence(seq, max_length):
    return seq + [0] * (max_length - len(seq))

# Determine max length for padding
max_length = max([max(len(seq) for seq in item[:4] if isinstance(seq, list)) for item in batch])

# Processing the batch and converting to JAX tensors
input_ids = jnp.array([pad_sequence(item[0], max_length) for item in batch], dtype=jnp.int32)
segment_ids = jnp.array([pad_sequence(item[1], max_length) for item in batch], dtype=jnp.int32)
masked_tokens = jnp.array([pad_sequence(item[2], max_length) for item in batch], dtype=jnp.int32)
masked_pos = jnp.array([pad_sequence(item[3], max_length) for item in batch], dtype=jnp.int32)
isNext = jnp.array([item[4] for item in batch], dtype=jnp.bool_)

In [56]:
# Generate the attention mask for the batch
attn_mask = get_attn_pad_mask(input_ids, input_ids)

# Output for debugging
print("Input IDs:\n", input_ids)
print("Segment IDs:\n", segment_ids)
print("Masked Tokens:\n", masked_tokens)
print("Masked Positions:\n", masked_pos)
print("Is Next:\n", isNext)
print("Attention Mask:\n", attn_mask[0][0])

Input IDs:
 [[  1  97  28 ...   0   0   0]
 [  1 202   3 ... 295  30   2]
 [  1 138 200 ...   0   0   0]
 [  1 236 194 ...   0   0   0]
 [  1 236 194 ...   0   0   0]
 [  1 138 200 ...   0   0   0]]
Segment IDs:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Masked Tokens:
 [[252  94 319 ...   0   0   0]
 [331 163 274 ...   0   0   0]
 [165 296 233 ...   0   0   0]
 [182 352  71 ...   0   0   0]
 [116  25 309 ...   0   0   0]
 [143 210 142 ...   0   0   0]]
Masked Positions:
 [[ 56  35  17 ...   0   0   0]
 [ 36   2 162 ...   0   0   0]
 [107  37   3 ...   0   0   0]
 [138 145 121 ...   0   0   0]
 [ 28  46  52 ...   0   0   0]
 [ 27  21  11 ...   0   0   0]]
Is Next:
 [False False False  True  True  True]
Attention Mask:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

# 6. Building Blocks
1. Embedding.
2. Multi Head Attention.
3. Feed Forward
4. Encoder Block

In [86]:
import jax.numpy as jnp
import flax.linen as nn
from jax import random
import numpy as np
# Initialize PRNG key
rng = random.PRNGKey(0)
class Embedding(nn.Module):
    vocab_size: int
    d_model: int
    maxlen: int
    n_segments: int

    @nn.compact
    def __call__(self, input_ids, segment_ids):
        tok_embed = nn.Embed(self.vocab_size, self.d_model)(input_ids)
        pos_embed = nn.Embed(self.maxlen, self.d_model)(jnp.arange(input_ids.shape[1])[None, :])
        seg_embed = nn.Embed(self.n_segments, self.d_model)(segment_ids)
        embedding = tok_embed + pos_embed + seg_embed
        return nn.LayerNorm()(embedding)

In [87]:
class ScaledDotProductAttention(nn.Module):
    d_k: int

    @nn.compact
    def __call__(self, Q, K, V, attn_mask):
        scores = jnp.einsum('bqhd,bkhd->bhqk', Q, K) / np.sqrt(self.d_k)
        scores = jnp.where(attn_mask == 0, scores, -1e9)  # Apply the attention mask
        attn = nn.softmax(scores, axis=-1)
        context = jnp.einsum('bhqk,bkhd->bqhd', attn, V)
        return context, attn

In [88]:
# Initialize Flax RNG
rng = random.PRNGKey(0)

# Example usage (input_ids, segment_ids need to be provided)
embedding_layer = Embedding(vocab_size=50000, d_model=512, maxlen=512, n_segments=2)

# Initialize the parameters
params = embedding_layer.init(rng, input_ids, segment_ids)

# Use the initialized parameters
embeds = embedding_layer.apply(params, input_ids, segment_ids)

attenM = get_attn_pad_mask(input_ids, input_ids)

attention_layer = ScaledDotProductAttention(d_k=64)
context, attn = attention_layer.apply({'params': {}}, embeds, embeds, embeds, attenM)

print('Masks:', attenM[0][0])
print()
print('Attention Scores after softmax: ', attn[0][0])

ValueError: Einstein sum subscript 'bqhd' does not contain the correct number of indices for operand 0.

In [89]:
import flax.linen as nn
import jax.numpy as jnp
import numpy as np

class MultiHeadAttention(nn.Module):
    d_model: int
    d_k: int
    d_v: int
    n_heads: int

    @nn.compact
    def __call__(self, Q, K, V, attn_mask):
        batch_size = Q.shape[0]
        q_s = nn.Dense(self.d_k * self.n_heads)(Q).reshape(batch_size, -1, self.n_heads, self.d_k)
        k_s = nn.Dense(self.d_k * self.n_heads)(K).reshape(batch_size, -1, self.n_heads, self.d_k)
        v_s = nn.Dense(self.d_v * self.n_heads)(V).reshape(batch_size, -1, self.n_heads, self.d_v)

        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s, attn_mask)
        context = context.reshape(batch_size, -1, self.d_v * self.n_heads)
        output = nn.Dense(self.d_model)(context)
        return nn.LayerNorm()(output + Q), attn

In [92]:
# Model instantiation
embedding_layer = Embedding(vocab_size=50000, d_model=512, maxlen=512, n_segments=2)
multihead_attention_layer = MultiHeadAttention(d_model=512, d_k=64, d_v=64, n_heads=8)

# Initialize parameters for Embedding
params_emb = embedding_layer.init(rng, input_ids, segment_ids)
# Apply embedding
embeds = embedding_layer.apply(params_emb, input_ids, segment_ids)

# Initialize parameters for MultiHeadAttention
params_mha = multihead_attention_layer.init(rng, embeds, embeds, embeds, attn_mask)

# Apply MultiHeadAttention
output, attn = multihead_attention_layer.apply(params_mha, embeds, embeds, embeds, attn_mask)

print("Output:", output)
print("Attention Weights:", attn)

ValueError: Incompatible shapes for broadcasting: shapes=[(6, 182, 182), (2, 8, 3, 3), ()]

In [ ]:
len(A[0][0])

In [ ]:
class PoswiseFeedForwardNet(jnn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = jnn.Linear(d_model, d_ff)
        self.fc2 = jnn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(gelu(self.fc1(x)))

In [ ]:
class EncoderLayer(jnn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

# 7. BERT

In [ ]:
class BERT(jnn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = jnn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = jnn.Linear(d_model, d_model)
        self.activ1 = jnn.Tanh()
        self.linear = jnn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = jnn.LayerNorm(d_model)
        self.classifier = jnn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = jnn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = jnn.Parameter(jax.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_model, d_model]
        # it will be decided by first token(CLS)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        # get masked position from final output of transformer.
        h_masked = jax.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_clsf

# 8. Train Our Model

In [ ]:
model = BERT()
criterion = jnn.CrossEntropyLoss()
optimizer = optax.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(jax.LongTensor, zip(*batch))

for epoch in range(500):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

# 9. Inference

In [ ]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(jax.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)